# Association Rule Learning Recommendation System

Birliktelik Kuralına ilişkin olarak bir kullanıcının okuduğu kitaplara ilişkin en yakın hangi kitapları tavsiye edebiliriz.

### Importing Necessary Libraries

In [1]:
#pip install mlxtend

In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 50)
import warnings
warnings.filterwarnings('ignore')

### Reading Datasets

In [3]:
df= pd.read_csv('df_final_21_09___9794.csv',index_col=0)
df

,book_id,book_title,book_series,title,book_authors,genres,book_score,book_rating,book_rating_obj,book_rating_count,book_review_count,book_desc,tags,FE_text,book_desc_tags_FE,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,book_edition,book_format,original_publication_year,language_code,book_pages,book_pages_obj,books_count,books_count_obj,goodreads_book_id,book_isbn,isbn,isbn13,image_url_x,image_url_y,small_image_url
0,1,The Hunger Games,"The Hunger Games, #1","The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Young Adult|Fiction|Science Fiction|Dystopia|F...,1.000000e+00,4.33,4.0,5519135,160706,Winning will make you famous. Losing means cer...,to-read fantasy favorites currently-reading yo...,The-Hunger-Games Suzanne-Collins Young-Adult F...,Winning will make you famous. Losing means cer...,66715,127936,560092,1481305,2706317,NaN,Hardcover,2008.0,eng,374.0,8.0,272,14.0,2767052,9.780440e+12,439023483,9.780439e+12,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603l...,https://images.gr-assets.com/books/1447303603s...
1,2,Harry Potter and the Philosopher's Stone,"Harry Potter, #1",Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",NaN,8.245693e-01,4.44,4.0,6682936,108361,NaN,to-read fantasy favorites currently-reading yo...,Harry-Potter-and-the-Philosopher's-Stone J.K.-...,to-read fantasy favorites currently-reading y...,75504,101676,455024,1156318,3011543,NaN,NaN,1997.0,eng,NaN,NaN,491,15.0,3,NaN,439554934,9.780440e+12,https://images.gr-assets.com/books/1474154022m...,NaN,https://images.gr-assets.com/books/1474154022s...
2,3,Twilight,"Twilight, #1","Twilight (Twilight, #1)",Stephenie Meyer,Young Adult|Fantasy|Romance|Paranormal|Vampire...,2.902011e-02,3.58,3.5,4281268,97991,About three things I was absolutely positive.F...,to-read fantasy favorites currently-reading yo...,Twilight Stephenie-Meyer Young-Adult Fantasy R...,About three things I was absolutely positive.F...,456191,436802,793319,875073,1355439,NaN,Paperback,2005.0,en-US,498.0,9.0,226,14.0,41865,9.780320e+12,316015849,9.780316e+12,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443l...,https://images.gr-assets.com/books/1361039443s...
3,4,To Kill a Mockingbird,-,To Kill a Mockingbird,Harper Lee,Classics|Fiction|Historical|Historical Fiction...,2.310594e-01,4.27,4.0,3745197,79450,The unforgettable novel of a childhood in a sl...,to-read favorites currently-reading young-adul...,To-Kill-a-Mockingbird Harper-Lee Classics Fict...,The unforgettable novel of a childhood in a sl...,60427,117415,446835,1001952,1714267,50th Anniversary,Paperback,1960.0,eng,324.0,8.0,487,15.0,2657,9.780060e+12,61120081,9.780061e+12,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680l...,https://images.gr-assets.com/books/1361975680s...
4,5,The Great Gatsby,-,The Great Gatsby,F. Scott Fitzgerald,Classics|Fiction|Academic|School|Literature|Hi...,4.942671e-02,3.90,3.5,3141842,56953,Alternate Cover Edition ISBN: 0743273567 (ISBN...,to-read favorites currently-reading young-adul...,The-Great-Gatsby F.-Scott-Fitzgerald Classics ...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,86236,197621,606158,936012,947718,US / CAN,Paperback,1925.0,eng,180.0,5.0,1356,15.0,4671,NaN,743273567,9.780743e+12,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560l...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,9996,Bayou Moon,"The Edge, #2","Bayou Moon (The Edge, #2)",Ilona Andrews,Fantasy|Urban Fantasy|Fantasy|Romance|Fantasy|...,5.206229e-06,4.10,4.0,21145,1310,"The Edge lies between worlds, on the border be...",to-read fantasy favorites currently-reading fi...,Bayou-Moon Ilona-Andrews Fantasy Urban-Fantasy...,"The Edge lies between worlds, on the bord

In [4]:
df_read = pd.read_csv('to_read.csv',index_col=0)
df_read=df_read.reset_index()
df_read.head(5)

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [5]:
df_ratings = pd.read_csv('ratings.csv',index_col=0)
df_ratings=df_ratings.reset_index()
df_ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


### Data Preprocessing

In [6]:
df_ratings["user_id"].value_counts()

12874    200
30944    200
52036    199
12381    199
28158    199
        ... 
32128     21
40753     21
51725     21
43675     20
34590     19
Name: user_id, Length: 53424, dtype: int64

Veri setinde 53424 farklı tipte kullanıcı mevcut. Bu kullanıcılar arasında belli sayının üzerindeki kullanıcıları seçerek veri setini indirgeriz ve daha efektif sonuçlar elde ederiz.

> **Burada tüm kullanıcıları ve kitapları seçmemiz gerekiyor. Ancak bunu yapmamız durumunda yaklaşık 800 milyar satırlık-100 GB üstü veri elde ediliyor. Bu veriyi bu ortamlarda elde etmemiz çok uzun sürüyor vede bu kadar alanıımız yok. bu nedenle bu kullanıcı ve kitap sayısını optimum sayıya azaltarak sonuçlarımızı göstermek istedik.Bu işlemler sonucunda 20 üzeri kitap okuyan tüm kullanıcılar için en beğenilen 500 kitap arasından önerilerde bulunmuş olduk.**

>20 üzeri kitap okuyan kullanıcıları seçtik

In [7]:
df_bookish_user=df_ratings["user_id"].value_counts()
df_bookish_user=df_bookish_user[df_bookish_user>20].index

>score değeri ortalama üzeri olan, 4.31 değer üzeri rating puanı olan 500 kitabı seçtik

In [8]:
df6=df[df["book_rating"]>4.31][df["book_score"]>df["book_score"].median()]
df6=df6.iloc[:500]
df_book_ids=df6["book_id"].values

Yukarıda oluşturduğumuz kullanıcıları ve kitapları daha önce tanımladığımız "df_read" veri setinde okutup yeni bir veri seti oluşturuyoruz.

In [9]:
df_read3=df_read[df_read["user_id"].isin(df_bookish_user)][df_read["book_id"].isin(df_book_ids)]

Yeni oluşturduğumuz "df_read3" veri setine kitap başlıklarımızı da ekleyip veri setini güncelliyoruz.

In [10]:
df_read3=df_read3.merge(df[["book_id","title"]],on="book_id") 
df_read3["Quantity"]=1
df_read3

,user_id,book_id,title,Quantity
0,207,25,Harry Potter and the Deathly Hallows (Harry Po...,1
1,590,25,Harry Potter and the Deathly Hallows (Harry Po...,1
2,656,25,Harry Potter and the Deathly Hallows (Harry Po...,1
3,779,25,Harry Potter and the Deathly Hallows (Harry Po...,1
4,1147,25,Harry Potter and the Deathly Hallows (Harry Po...,1
...,...,...,...,...
70526,19344,4641,Fantastic Beasts and Where to Find Them: The O...,1
70527,31724,4641,Fantastic Beasts and Where to Find Them: The O...,1
70528,29044,4641,Fantastic Beasts and Where to Find Them: The O...,1
70529,27692,4641,Fantastic Beasts and Where to Find Them: The O...,1


In [11]:
df_read3["user_id"].nunique()

27772

In [12]:
df_read3["book_id"].nunique()

500

> Yapılan işlem sonrasında **500 kitap ve 27772 kullanıcı**mız mevcuttur.

# Association Rule Learning(Birliktelik Kuralı Öğrenimi)

> Veritabanları içerisinde ürünler arasındaki ilişkiyi ortaya çıkarmak için kullanılan temel tekniklerden biridir. Başka bir deyişle, birlikte daha sık satın alınan ürünler arasındaki ilişkiyi belirlemeyi sağlar.

 >Bu kural şu adlandırmalar ile de adlandırılmıştır: Market Sepet Analizi (Market Basket Analysis), Birliktelik Kuralları (Association Rules), Tavsiye Sistemleri (Recommended Systems)

**Define  Matrix Function**

In [13]:
def create_matrix_title_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['user_id', "book_id"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['user_id', 'title'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
# burada seçimimiz sonrası "book_title" veya 'book_id'a göre bize matriks oluşturacak

In [14]:
df_ar_title = create_matrix_title_df(df_read3)
df_ar_title

title,"A Breath of Snow and Ashes (Outlander, #6)","A Clash of Kings (A Song of Ice and Fire, #2)","A Court of Mist and Fury (A Court of Thorns and Roses, #2)","A Court of Wings and Ruin (A Court of Thorns and Roses, #3)","A Dance with Dragons (A Song of Ice and Fire, #5)","A Dangerous Path (Warriors, #5)","A Dog's Journey (A Dog's Purpose, #2)","A Dog's Purpose (A Dog's Purpose, #1)",A Fine Balance,"A Game of Thrones (A Song of Ice and Fire, #1)","A Game of Thrones: The Graphic Novel, Vol. 1",A Game of You (The Sandman #5),"A Gathering of Shadows (Shades of Magic, #2)",A Gentleman in Moscow,"A Great Reckoning (Chief Inspector Armand Gamache, #12)",A Light in the Attic,A Man Called Ove,"A Memory of Light (Wheel of Time, #14)",A Monster Calls,"A Perfect Blood (The Hollows, #10)","A Song of Ice and Fire (A Song of Ice and Fire, #1-4)","A Song of Ice and Fire (A Song of Ice and Fire, #1-5)","A Storm of Swords (A Song of Ice and Fire, #3)","A Storm of Swords: Blood and Gold (A Song of Ice and Fire, #3: Part 2 of 2)","A Storm of Swords: Steel and Snow (A Song of Ice and Fire, #3: Part 1 of 2)",...,"Unbroken: A World War II Story of Survival, Resilience, and Redemption","United We Spy (Gallagher Girls, #6)","Unleash the Night (Dark Hunter, #8; Were-Hunter, #2)","Until You (Fall Away, #1.5)","Vengeance in Death (In Death, #6)","Voyager (Outlander, #3)","War and Remembrance (The Henry Family, #2)",Watchmen,We Should All Be Feminists,"Wethering the Storm (The Storm, #2)",When Breath Becomes Air,"Whispers at Moonrise (Shadow Falls, #4)","White Night (The Dresden Files, #9)","White Witch, Black Curse (The Hollows, #7)",Wicked Deeds on a Winter's Night (Immortals After Dark #4),"Wild Magic (Immortals, #1)","Winnie-the-Pooh (Winnie-the-Pooh, #1)","Winter (The Lunar Chronicles, #4)",With the Old Breed: At Peleliu and Okinawa,Wonder,"Words of Radiance (The Stormlight Archive, #2)",Worlds' End (The Sandman #8),"Written in My Own Heart's Blood (Outlander, #8)","Y: The Last Man, Vol. 10: Whys and Wherefores (Y: The Last Man, #10)","You Are Special (Wemmicksville, #1)"
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53417,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53422,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Kullanıcı bazında kitap başlıklarına göre matrix veri seti tanımlıyoruz.**

> book_id sini bildiğimiz kitabın adını getiren fonksiyon tanımladık

In [15]:
def check_id(dataframe, book_id):
    product_name = dataframe[dataframe["book_id"] == book_id][["title"]].values[0].tolist()
    print(product_name)


check_id(df, 1)

['The Hunger Games (The Hunger Games, #1)']


## Birliktelik Kurallarının Çıkarılması

* 'antecedents',        ilk ürün
* 'consequents',        ikinci ürün
* 'antecedent support', ürünün tek başına gözlenme olasılığı
* 'consequent support', ürünün tek başına gözlenme olasılığı
* 'support',            ürünlerin birlikte gözlenme olasılığı
* 'confidence',         X alındığında Y'nin alınma olasılığı
* 'lift',               X alındığında Y'nin alınma olasılığı ... kat artar
* 'leverage',     kaldıraç etkisi demek. lift'e benzer bir değerdir Fakat sapportu yüksek olanlara öncelik verme eğilimindedir. Bu nedenle biraz yanlıdır. Lift ise daha az sıklıkta olmasına rağmen ilişkileri yakalayabilmektedir. 
* 'conviction'    Y/X ürünü olmadan X/Y'in beklenen frekansıdır

In [16]:
# min_support=0.002 değerine göre seçtiğimiz kitaplar arasında anlamlı ilişki olan 553 kitap buldu.
# daha büyük RAM ve database ortamında min_support değeri küçültülerek daha büyük ilişki ağı ortaya çıkarılabilir. 
# burada en önemli olan 553 ilişkiyi tespit etti.

# Apriori ilkesi, destek için minimum eşik koşulunu karşılamayan bir öğe kümesinin tüm üst kümelerini budamamıza izin verir. 
# Örneğin, {A, B}, minimum destek eşiğimizi karşılamıyorsa, 
# buna herhangi bir öğe eklenmiş bir öğe kümesi de eşiği asla geçmeyecektir.

In [17]:
frequent_itemsets = apriori(df_ar_title,
                            min_support=0.002,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
166,0.099813,(The Book Thief)
20,0.070827,(All the Light We Cannot See)
7,0.058296,"(A Game of Thrones (A Song of Ice and Fire, #1))"
199,0.051023,(The Help)
17,0.048682,(A Thousand Splendid Suns)
...,...,...
295,0.002016,"(All the Light We Cannot See, Between Shades o..."
235,0.002016,(The War that Saved My Life (The War That Save...
413,0.002016,"(Dragonfly in Amber (Outlander, #2), Written i..."
523,0.002016,"(Drums of Autumn (Outlander, #4), An Echo in t..."


In [18]:
# frequent_itemsets öğe kümelerinden tüm olası kuralları oluşturma

In [19]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.000001)
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(An Echo in the Bone (Outlander, #7))","(A Breath of Snow and Ashes (Outlander, #6))",0.010262,0.009866,0.006733,0.656140,66.504853,0.006632,2.879471
1,"(A Breath of Snow and Ashes (Outlander, #6))","(An Echo in the Bone (Outlander, #7))",0.009866,0.010262,0.006733,0.682482,66.504853,0.006632,3.117105
2,"(Dragonfly in Amber (Outlander, #2))","(A Breath of Snow and Ashes (Outlander, #6))",0.011234,0.009866,0.003313,0.294872,29.887516,0.003202,1.404190
3,"(A Breath of Snow and Ashes (Outlander, #6))","(Dragonfly in Amber (Outlander, #2))",0.009866,0.011234,0.003313,0.335766,29.887516,0.003202,1.488581
4,"(Drums of Autumn (Outlander, #4))","(A Breath of Snow and Ashes (Outlander, #6))",0.009254,0.009866,0.005221,0.564202,57.186231,0.005130,2.272004
...,...,...,...,...,...,...,...,...,...
2639,(Harry Potter and the Half-Blood Prince (Harry...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015771,0.002449,0.002376,0.150685,61.541499,0.002338,1.174536
2640,(Harry Potter and the Order of the Phoenix (Ha...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015375,0.002521,0.002376,0.154567,61.323252,0.002338,1.179844
2641,(Harry Potter and the Prisoner of Azkaban (Har...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015411,0.002485,0.002376,0.154206,62.066640,0.002338,1.179383
2642,(Harry Potter and the Chamber of Secrets (Harr...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.014691,0.002485,0.002376,0.161765,65.109122,0.002340,1.190018


In [20]:
rules["antecedents"].nunique()

275

Burada birliktelik kuralına ilişkin olarak "Support", "Confidence" ve "Lift" değerlerini belli değerlerin üzerinde seçiyoruz ve "confidence" değerine göre büyükten küçüğe sıralıyoruz.

In [21]:
rules[(rules["support"]>0.002) & (rules["confidence"]>0.1) & (rules["lift"]>50)]. \
sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
833,"(Dragonfly in Amber (Outlander, #2), Written i...","(An Echo in the Bone (Outlander, #7))",0.002052,0.010262,0.002052,1.000000,97.445614,0.002031,inf
1427,"(Drums of Autumn (Outlander, #4), Written in M...","(An Echo in the Bone (Outlander, #7))",0.002016,0.010262,0.002016,1.000000,97.445614,0.001996,inf
1425,"(Drums of Autumn (Outlander, #4), Written in M...","(Voyager (Outlander, #3))",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
804,"(Drums of Autumn (Outlander, #4), An Echo in t...","(Voyager (Outlander, #3))",0.003061,0.009542,0.003025,0.988235,103.567059,0.002995,84.188931
1365,"(Drums of Autumn (Outlander, #4), A Breath of ...","(Voyager (Outlander, #3))",0.002809,0.009542,0.002773,0.987179,103.456410,0.002746,77.255725
...,...,...,...,...,...,...,...,...,...
2204,(Harry Potter and the Half-Blood Prince (Harry...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015771,0.002557,0.002485,0.157534,61.620297,0.002444,1.183957
2640,(Harry Potter and the Order of the Phoenix (Ha...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015375,0.002521,0.002376,0.154567,61.323252,0.002338,1.179844
2641,(Harry Potter and the Prisoner of Azkaban (Har...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015411,0.002485,0.002376,0.154206,62.066640,0.002338,1.179383
2643,(Harry Potter and the Goblet of Fire (Harry Po...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.015519,0.002485,0.002376,0.153132,61.634621,0.002338,1.177888


In [22]:
sorted_rules = rules.sort_values("confidence", ascending=False)
# burada lift,confidence veya support' a göre sıralama yapılabilir. Hangisini önemli görüyor isek.
sorted_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1425,"(Drums of Autumn (Outlander, #4), Written in M...","(Voyager (Outlander, #3))",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
1427,"(Drums of Autumn (Outlander, #4), Written in M...","(An Echo in the Bone (Outlander, #7))",0.002016,0.010262,0.002016,1.000000,97.445614,0.001996,inf
833,"(Dragonfly in Amber (Outlander, #2), Written i...","(An Echo in the Bone (Outlander, #7))",0.002052,0.010262,0.002052,1.000000,97.445614,0.002031,inf
804,"(Drums of Autumn (Outlander, #4), An Echo in t...","(Voyager (Outlander, #3))",0.003061,0.009542,0.003025,0.988235,103.567059,0.002995,84.188931
1365,"(Drums of Autumn (Outlander, #4), A Breath of ...","(Voyager (Outlander, #3))",0.002809,0.009542,0.002773,0.987179,103.456410,0.002746,77.255725
...,...,...,...,...,...,...,...,...,...
57,(The Book Thief),"(A Storm of Swords (A Song of Ice and Fire, #3))",0.099813,0.020416,0.002232,0.022367,1.095526,0.000195,1.001995
249,(The Book Thief),(Wonder),0.099813,0.015771,0.002160,0.021645,1.372433,0.000586,1.006004
216,(The Book Thief),(Man's Search for Meaning),0.099813,0.017212,0.002088,0.020924,1.215665,0.000370,1.003791
245,(The Book Thief),(The Stand),0.099813,0.019732,0.002052,0.020563,1.042097,0.000083,1.000848


In [23]:
# Kullanıcı A ve E kitaplarını okumuş olsun.

# Burada {A,B}-{C,D} kümeleri arasında ilişki var ise,
#    {A,C},
#    {A,D},
#    {B,C},
#    {B,D}  arasında ilişki vardır.

# ek olarak  {E,F}-{C,D} kümeleri arasında ilişki var ise,
#    {E,C},
#    {E,D},
#    {F,C},
#    {F,D}  arasında ilişki vardır.

# Bu ilişkileri ikinci sutun ile groupby yaptığımızda kullanıcının okuduğu A,E ile en ilişkili kitabın C olduğunu bulmayı 
# amaçlıyoruz.
# Bunu yapabilmek için kitaplar arasında tekli ilişkiyi gösteren aşağıdaki işlemi uyguluyoruz.

# İstenmesi durumunda bu çalışma ile standart yapılan çalışmalar gibi tek bir kitap için de tavsiyede bulunulabilir.

In [24]:
dfx=sorted_rules.copy()
cols = ['antecedents','consequents']
dfy=dfx[cols].applymap(lambda x: tuple(x))
dfy=pd.concat([dfy,dfx[dfx.columns[2:]]],axis=1)
dfy = (dfy.explode('antecedents')
         .reset_index(drop=True)
         .explode('consequents')
         .reset_index(drop=True))
dfy

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
1,"Written in My Own Heart's Blood (Outlander, #8)","Voyager (Outlander, #3)",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
2,"An Echo in the Bone (Outlander, #7)","Voyager (Outlander, #3)",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
3,"Dragonfly in Amber (Outlander, #2)","Voyager (Outlander, #3)",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
4,"Drums of Autumn (Outlander, #4)","An Echo in the Bone (Outlander, #7)",0.002016,0.010262,0.002016,1.000000,97.445614,0.001996,inf
...,...,...,...,...,...,...,...,...,...
11931,The Book Thief,"A Storm of Swords (A Song of Ice and Fire, #3)",0.099813,0.020416,0.002232,0.022367,1.095526,0.000195,1.001995
11932,The Book Thief,Wonder,0.099813,0.015771,0.002160,0.021645,1.372433,0.000586,1.006004
11933,The Book Thief,Man's Search for Meaning,0.099813,0.017212,0.002088,0.020924,1.215665,0.000370,1.003791
11934,The Book Thief,The Stand,0.099813,0.019732,0.002052,0.020563,1.042097,0.000083,1.000848


In [25]:
# İlgili kitapların book_id değerlerini buluyoruz 

In [26]:
dfy=dfy.merge(df[["title","book_id"]], left_on="antecedents",right_on="title")
dfy = dfy.rename(columns={'book_id': 'book_id_antecedents'})

dfy=dfy.merge(df[["title","book_id"]], left_on="consequents",right_on="title")
dfy = dfy.rename(columns={'book_id': 'book_id_consequents'})

dfy=dfy[['book_id_antecedents', 'book_id_consequents','antecedents', 'consequents',
       'antecedent support','consequent support', 'support','confidence', 'lift', 'leverage','conviction' ]]
dfy

,book_id_antecedents,book_id_consequents,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,712,554,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.002016,0.009542,0.002016,1.000000,104.800000,0.001997,inf
1,712,554,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.003061,0.009542,0.003025,0.988235,103.567059,0.002995,84.188931
2,712,554,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.002809,0.009542,0.002773,0.987179,103.456410,0.002746,77.255725
3,712,554,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.002052,0.003817,0.002016,0.982456,257.403509,0.002009,56.782443
4,712,554,"Drums of Autumn (Outlander, #4)","Voyager (Outlander, #3)",0.002052,0.009542,0.002016,0.982456,102.961404,0.001997,56.456107
...,...,...,...,...,...,...,...,...,...,...,...
11931,143,983,All the Light We Cannot See,Between the World and Me,0.070827,0.013791,0.002016,0.028470,2.064391,0.001040,1.015109
11932,143,1192,All the Light We Cannot See,Between Shades of Gray,0.070827,0.016023,0.002016,0.028470,1.776768,0.000882,1.012811
11933,47,250,The Book Thief,Wonder,0.099813,0.015771,0.002160,0.021645,1.372433,0.000586,1.006004
11934,47,472,The Book Thief,Man's Search for Meaning,0.099813,0.017212,0.002088,0.020924,1.215665,0.000370,1.003791


In [27]:
# Burada birbiri arasında ilişki olan kitap grublarından birini görüyoruz

In [28]:
sorted_rules["antecedents"].iloc[0]

frozenset({'An Echo in the Bone (Outlander, #7)',
           'Dragonfly in Amber (Outlander, #2)',
           'Drums of Autumn (Outlander, #4)',
           "Written in My Own Heart's Blood (Outlander, #8)"})

In [29]:
sorted_rules["consequents"].iloc[0]

frozenset({'Voyager (Outlander, #3)'})

In [30]:
# bir kitabı okuyan kullanıcıları görüyoruz

In [31]:
df_read3[df_read3["title"]=="Clockwork Angel (The Infernal Devices, #1)"]

,user_id,book_id,title,Quantity
34642,7241,183,"Clockwork Angel (The Infernal Devices, #1)",1
34643,9430,183,"Clockwork Angel (The Infernal Devices, #1)",1
34644,10099,183,"Clockwork Angel (The Infernal Devices, #1)",1
34645,4011,183,"Clockwork Angel (The Infernal Devices, #1)",1
34646,8068,183,"Clockwork Angel (The Infernal Devices, #1)",1
...,...,...,...,...
35494,50571,183,"Clockwork Angel (The Infernal Devices, #1)",1
35495,48721,183,"Clockwork Angel (The Infernal Devices, #1)",1
35496,50609,183,"Clockwork Angel (The Infernal Devices, #1)",1
35497,48384,183,"Clockwork Angel (The Infernal Devices, #1)",1


In [32]:
# Örnek olarak seçtiğimiz bir kitaba ait ilişki ağını görüyoruz

In [33]:
product_id = 25
check_id(df, product_id)

['Harry Potter and the Deathly Hallows (Harry Potter, #7)']


In [34]:
product_name=frozenset({'Harry Potter and the Deathly Hallows (Harry Potter, #7)'})

In [35]:
# sırasıyla tavsiye edilenler
recommendation_list=sorted_rules[sorted_rules["antecedents"]==product_name]#.iloc[:3,1]#.values
recommendation_list

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
181,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Half-Blood Prince (Harry...,0.017428,0.015771,0.011126,0.638430,40.480528,0.010851,2.722095
183,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Order of the Phoenix (Ha...,0.017428,0.015375,0.009686,0.555785,36.148160,0.009418,2.216551
179,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Goblet of Fire (Harry Po...,0.017428,0.015519,0.009146,0.524793,33.815689,0.008875,2.071690
582,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Half-Blood Prince (Harry...,0.017428,0.010622,0.008822,0.506198,47.654714,0.008637,2.003594
185,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Prisoner of Azkaban (Har...,0.017428,0.015411,0.008678,0.497934,32.309859,0.008409,1.961074
...,...,...,...,...,...,...,...,...,...
2513,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.017428,0.002989,0.002485,0.142562,47.701583,0.002432,1.162780
2389,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.017428,0.002953,0.002485,0.142562,48.283310,0.002433,1.162822
2203,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.017428,0.002917,0.002485,0.142562,48.879400,0.002434,1.162864
2327,(Harry Potter and the Deathly Hallows (Harry P...,(Harry Potter and the Sorcerer's Stone (Harry ...,0.017428,0.002953,0.002449,0.140496,47.583552,0.002397,1.160026


# Kullanıcının okuduğu tüm kitapları dikkate alarak ARL ile kitap tavsiyesi

* ........................................şu kitaplara en yakın kitap bul (en iyi 500 kitap içerisinden)---denilerek tavsiye sistemi geliştirilebilir
* Buradan seçilen kitaplar  "okudugu_kitaplar" a atanır.Okudugu_kitaplar="Kullanıcının 'şu kitablara en yakın kitap bul' dan seçtiği kitapplar"


In [36]:
# Örnek olarak bir kullanıcı seçiyoruz

In [37]:
user=df_bookish_user[0]
user

12874

In [38]:
# Okuduğu kitapları buluyoruz

In [39]:
okudugu_kitaplar=df_ratings[df_ratings["user_id"]==user]["book_id"].values
okudugu_kitaplar

array([ 306,  118,   43,  943, 1186,  715,  138,  301,   93,  560,   79,
        341,   15,   66,   42,   75, 1457,   22,  142,   33,   57,    8,
         11,    4,    5,   24,   67,   21,  115,   26,   81,  407,    9,
        111,  331,  616,  216,  344,  383, 4793, 3406,  414,  465,  333,
       1685,  145, 1150,  955,  228, 1534,  150,   10,  265,   13,  131,
        501,   48,  233,  121,  916,   14,  230,  310,   63,  343,  178,
        650,  172,   55, 1714,  177,  171,  485,  396,  101,  714,  160,
        597,   83,   56,   85,   18,  133,  229,  117,  109,   29,  102,
         59,   58,  219,  125,   23,   76,  113,  179,  157,  498,   50,
        514,  130,  204,  321,  154,  116,   44,  296,  740, 1167,  213,
       1214,  444,  401,  427,  364,  264,  107,  959,  819,  358,  245,
        964,  984,  734,  877,  595,  278, 2142,  570,  621,  531, 2297,
        545,  471, 1099,  872, 1561, 1681,  479,  579,  402,  257, 1084,
       1015, 1280,  824, 2336,  497, 2236,    2,  2

In [40]:
# Okuduğu kitaplar ile ilişkisi olan ama daha önce hiç okumadığı kitaplar arasından, en büyük ilişki olan kitapları 
# önem sırasına göre sıralıyoruz. 

In [41]:
dfz=dfy[dfy["book_id_antecedents"].isin(okudugu_kitaplar)][~(dfy["book_id_consequents"].isin(okudugu_kitaplar))]
dfz=dfz.groupby("consequents").sum().sort_values("support",ascending=False)
dfz

,book_id_antecedents,book_id_consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
consequents,,,,,,,,,
"Harry Potter and the Half-Blood Prince (Harry Potter, #6)",21384,32805,8.366844,9.843475,4.402384,764.996345,100522.336718,4.341809,7105.842974
"Harry Potter and the Deathly Hallows (Harry Potter, #7)",21384,30375,8.581521,9.737325,4.223354,703.079341,97450.260764,4.162126,3805.283389
"The Fellowship of the Ring (The Lord of the Rings, #1)",1137,133,0.079973,0.149575,0.037772,3.466151,243.207028,0.036076,14.402987
The Book Thief,256,94,0.063913,0.199626,0.010622,0.330280,3.308999,0.004243,2.156512
The Help,67,31,0.048682,0.051023,0.004753,0.097633,1.913527,0.002269,1.051654
All the Light We Cannot See,67,143,0.048682,0.070827,0.003997,0.082101,1.159175,0.000549,1.012282
"A Game of Thrones (A Song of Ice and Fire, #1)",67,39,0.048682,0.058296,0.003673,0.075444,1.294148,0.000835,1.018547
East of Eden,67,225,0.048682,0.040004,0.003133,0.064349,1.608554,0.001185,1.026019
"Unbroken: A World War II Story of Survival, Resilience, and Redemption",67,144,0.048682,0.031795,0.002268,0.046598,1.465583,0.000721,1.015527


**Sonuç**

In [42]:
# Burada bulduğumuz sonuç tek bir kitaba/ayrıntıya bakmaktan ziyade genel resime bakıp bu kişinin ilgi alanına giren
# tüm kitapların belirlenmesidir. Ve bunlar arasında en öne çıkanı tavsiye etmektir. 
# Dilenirse(tavsiyemiz de bu yönde) bunlar arasından matrix factorization yaparak en yüksek puan vereceği ve de 
# o kişi ile en alakalı kitap tavsiye etmek çok mantıklıca olacaktır.

### Yeni Bir Kullanıcı Giriş Yapalım

In [43]:
user1=df_bookish_user[4]
user1

28158

In [44]:
okudugu_kitaplar=df_ratings[df_ratings["user_id"]==user1]["book_id"].values

In [45]:
dfz=dfy[dfy["book_id_antecedents"].isin(okudugu_kitaplar)][~(dfy["book_id_consequents"].isin(okudugu_kitaplar))]
dfz=dfz.groupby("consequents").sum().sort_values("support",ascending=False)
dfz

,book_id_antecedents,book_id_consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
consequents,,,,,,,,,
"Harry Potter and the Chamber of Secrets (Harry Potter, #2)",27945,27945,9.383300,8.808800,4.573275,708.580141,101801.795778,4.512092,4967.864526
"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",27945,2430,10.911422,6.065534,3.267248,391.401524,96903.130208,3.216075,1804.335233
The Book Thief,1686,564,0.379339,1.197753,0.046162,1.421708,14.243754,0.008299,12.264778
"The Hero of Ages (Mistborn, #3)",2862,3618,0.064958,0.032119,0.015699,2.914519,544.732043,0.015367,22.340485
All the Light We Cannot See,487,572,0.187131,0.283307,0.012603,0.279226,3.942386,-0.000651,3.996532
A Thousand Splendid Suns,295,201,0.149323,0.146046,0.011558,0.234464,4.816228,0.004289,3.096466
"The Way of Kings (The Stormlight Archive, #1)",581,1124,0.061177,0.038888,0.006229,0.217134,11.167124,0.005040,2.202216
"Clockwork Angel (The Infernal Devices, #1)",70,366,0.109319,0.061717,0.005617,0.101639,3.293715,0.002244,2.042222
"The Wise Man's Fear (The Kingkiller Chronicle, #2)",192,307,0.037808,0.012315,0.004573,0.120952,9.821899,0.004107,1.123586


### Yeni Bir Kullanıcı Giriş Yapalım

In [46]:
user2=df_bookish_user[1]
user2

30944

In [47]:
okudugu_kitaplar=df_ratings[df_ratings["user_id"]==user2]["book_id"].values

In [48]:
dfz=dfy[dfy["book_id_antecedents"].isin(okudugu_kitaplar)][~(dfy["book_id_consequents"].isin(okudugu_kitaplar))]
dfz=dfz.groupby("consequents").sum().sort_values("support",ascending=False)
dfz

,book_id_antecedents,book_id_consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
consequents,,,,,,,,,
"A Dance with Dragons (A Song of Ice and Fire, #5)",2556,5076,0.513827,0.484769,0.154436,13.475079,945.247683,0.146482,71.501387
The Book Thief,1241,376,0.223282,0.798502,0.026033,0.929526,9.312694,0.003747,8.153974
"The Titan's Curse (Percy Jackson and the Olympians, #3)",1002,1284,0.045585,0.038636,0.023225,3.233731,509.076917,0.022939,14.275138
"The Hero of Ages (Mistborn, #3)",1167,1809,0.049366,0.016671,0.007238,1.087806,179.931555,0.006981,10.283766
"The Well of Ascension (Mistborn, #2)",1167,1695,0.049366,0.016815,0.007238,1.087806,178.216707,0.006979,10.283101
The Help,58,62,0.085698,0.102045,0.006697,0.163633,3.207076,0.002325,2.067330
All the Light We Cannot See,231,286,0.096104,0.141653,0.006409,0.140738,1.987078,-0.000397,1.999704
"The Way of Kings (The Stormlight Archive, #1)",581,1124,0.061177,0.038888,0.006229,0.217134,11.167124,0.005040,2.202216
A Thousand Splendid Suns,39,67,0.058296,0.048682,0.003673,0.063002,1.294148,0.000835,1.015283


### "Kullanıcının ARL mantığına göre şu kitaba  en yakın kitabı bul" isteğini aşağıdaki gibi yapabiliriz

In [49]:
product_id = 19
check_id(df, product_id)

['The Fellowship of the Ring (The Lord of the Rings, #1)']


In [50]:
product_name= "The Fellowship of the Ring (The Lord of the Rings, #1)"

In [51]:
product_name=frozenset({"The Fellowship of the Ring (The Lord of the Rings, #1)"})

In [52]:
recommendation_list=sorted_rules[sorted_rules["antecedents"]==product_name]#.iloc[:3,1]#.values
recommendation_list

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
256,(The Fellowship of the Ring (The Lord of the R...,"(The Two Towers (The Lord of the Rings, #2))",0.027402,0.011594,0.006373,0.232589,20.060414,0.006056,1.287974
254,(The Fellowship of the Ring (The Lord of the R...,(The Return of the King (The Lord of the Rings...,0.027402,0.011882,0.006193,0.226018,19.021160,0.005868,1.276668
683,(The Fellowship of the Ring (The Lord of the R...,(The Return of the King (The Lord of the Rings...,0.027402,0.008894,0.005581,0.203679,22.901148,0.005337,1.244607
42,(The Fellowship of the Ring (The Lord of the R...,"(A Game of Thrones (A Song of Ice and Fire, #1))",0.027402,0.058296,0.003889,0.141919,2.434442,0.002291,1.097453
228,(The Fellowship of the Ring (The Lord of the R...,(The Book Thief),0.027402,0.099813,0.003493,0.127464,1.277030,0.000758,1.031690
253,(The Fellowship of the Ring (The Lord of the R...,"(The Lord of the Rings (The Lord of the Rings,...",0.027402,0.015231,0.002881,0.105125,6.901955,0.002463,1.100454
250,(The Fellowship of the Ring (The Lord of the R...,(The Help),0.027402,0.051023,0.002521,0.091984,1.802813,0.001122,1.045111
